In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# -------------------------------------------------------
# 1. VISUALIZATION: THE DATA GAP (Mirissa Grid)
# -------------------------------------------------------
def plot_grid_gap():
    # Load Metadata
    with open("model_metadata.pkl", "rb") as f:
        meta = pickle.load(f)
    
    lats = meta['latitude']
    lons = meta['longitude']
    
    # Mirissa Coordinates
    TARGET_LAT = 5.949
    TARGET_LON = 80.452
    VIRTUAL_BUOY_LAT = 5.5
    VIRTUAL_BUOY_LON = 80.5

    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.set_extent([80.2, 80.7, 5.4, 6.2])
    
    # Add Coastlines
    ax.add_feature(cfeature.COASTLINE, resolution='10m')
    ax.add_feature(cfeature.LAND, color='lightgray')
    ax.add_feature(cfeature.OCEAN, color='lightblue')

    # Plot ERA5 Grid Points
    # Create meshgrid
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    ax.scatter(lon_grid, lat_grid, color='black', s=30, alpha=0.5, label='ERA5 Reanalysis Grid (25km)', transform=ccrs.PlateCarree())

    # Plot Surf Spot
    ax.plot(TARGET_LON, TARGET_LAT, marker='*', color='red', markersize=15, label='Mirissa Surf Point', transform=ccrs.PlateCarree())

    # Plot Virtual Buoy
    ax.plot(VIRTUAL_BUOY_LON, VIRTUAL_BUOY_LAT, marker='^', color='blue', markersize=12, label='Virtual Buoy (Nearest Valid Ocean)', transform=ccrs.PlateCarree())

    # Draw Arrow
    plt.arrow(VIRTUAL_BUOY_LON, VIRTUAL_BUOY_LAT, TARGET_LON-VIRTUAL_BUOY_LON, TARGET_LAT-VIRTUAL_BUOY_LAT, 
              head_width=0.02, length_includes_head=True, color='blue', linestyle='--')

    plt.title("The Nearshore Data Gap: Mirissa", fontsize=16)
    plt.legend(loc='lower right')
    plt.savefig("slide_visual_1_grid_gap.png", dpi=300)
    print("✅ Generated 'slide_visual_1_grid_gap.png'")

# -------------------------------------------------------
# 2. VISUALIZATION: PHYSICS DOMAIN (Arugam Bathymetry)
# -------------------------------------------------------
def plot_bathymetry():
    # Load your GEBCO file
    try:
        ds = xr.open_dataset("backend/app/surfspots/arugambay/GEBCO_Arugambay.nc")
        elev = ds['elevation']
    except:
        print("⚠️ GEBCO file not found, skipping bathymetry plot.")
        return

    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot contours
    # We only care about negative values (underwater)
    elev.plot(ax=ax, cmap='DeepSea', vmin=-1000, vmax=0, add_colorbar=True, cbar_kwargs={'label': 'Depth (m)'})
    
    # Mark Boundaries
    # Arugam Grid Box
    ax.set_title("SWAN Physics Domain: Arugam Bay Bathymetry", fontsize=16)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    
    # Annotate
    ax.text(81.845, 6.84, "Surf Spot", color='white', fontweight='bold')
    ax.text(81.99, 6.94, "Boundary Input", color='yellow', fontweight='bold')

    plt.savefig("slide_visual_2_bathymetry.png", dpi=300)
    print("✅ Generated 'slide_visual_2_bathymetry.png'")

if __name__ == "__main__":
    try:
        plot_grid_gap()
        plot_bathymetry()
    except Exception as e:
        print(f"Error generating plots: {e}")

Error generating plots: [Errno 2] No such file or directory: 'model_metadata.pkl'
